In [1]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Подготовка датасета для загрузки в сеть

In [2]:
base_df = pd.read_csv('data/clean_frame.csv', index_col='ind')
base_df.columns

Index([&#39;ID&#39;, &#39;Код_группы&#39;, &#39;Год_Поступления&#39;, &#39;Год_Окончания_УЗ&#39;, &#39;Пособие&#39;,
       &#39;Общежитие&#39;, &#39;Наличие_Матери&#39;, &#39;Наличие_Отца&#39;, &#39;Опекунство&#39;, &#39;Село&#39;,
       &#39;Иностранец&#39;, &#39;КодФакультета&#39;, &#39;СрБаллАттестата&#39;, &#39;Статус&#39;,
       &#39;vacation_status&#39;, &#39;non_vacation_status&#39;, &#39;male&#39;, &#39;female&#39;, &#39;sex_summ&#39;,
       &#39;birth_year_int&#39;, &#39;basis&#39;, &#39;language&#39;, &#39;country&#39;, &#39;region&#39;, &#39;city&#39;,
       &#39;parents_country&#39;],
      dtype=&#39;object&#39;)

### Проверка и удаление отрицательных значений

In [3]:
base_df.isna().any().all()

False

In [4]:
base_df = base_df.fillna(0)

In [5]:
drop_col = ['Статус', 'vacation_status', 'non_vacation_status']
control_col = 'vacation_status'

In [6]:
all_columns_list = base_df.columns.to_list()
all_columns_list.remove('ID')
for col in drop_col:
    all_columns_list.remove(col)
all_columns_list

[&#39;Код_группы&#39;,
 &#39;Год_Поступления&#39;,
 &#39;Год_Окончания_УЗ&#39;,
 &#39;Пособие&#39;,
 &#39;Общежитие&#39;,
 &#39;Наличие_Матери&#39;,
 &#39;Наличие_Отца&#39;,
 &#39;Опекунство&#39;,
 &#39;Село&#39;,
 &#39;Иностранец&#39;,
 &#39;КодФакультета&#39;,
 &#39;СрБаллАттестата&#39;,
 &#39;male&#39;,
 &#39;female&#39;,
 &#39;sex_summ&#39;,
 &#39;birth_year_int&#39;,
 &#39;basis&#39;,
 &#39;language&#39;,
 &#39;country&#39;,
 &#39;region&#39;,
 &#39;city&#39;,
 &#39;parents_country&#39;]

In [7]:
bad_columns = []

In [8]:
elements_num = len(all_columns_list)
elements_num = 2

In [9]:
all_combinations = []
list_sizes = 0
min_col = 2
for i in range(min_col, elements_num+1):
    curent_list = list(combinations(all_columns_list, i))
    list_sizes += len(curent_list)
    all_combinations = all_combinations + curent_list
comb_size = len(all_combinations)
comb_size

231

In [10]:
all_combinations

[(&#39;Код_группы&#39;, &#39;Год_Поступления&#39;),
 (&#39;Код_группы&#39;, &#39;Год_Окончания_УЗ&#39;),
 (&#39;Код_группы&#39;, &#39;Пособие&#39;),
 (&#39;Код_группы&#39;, &#39;Общежитие&#39;),
 (&#39;Код_группы&#39;, &#39;Наличие_Матери&#39;),
 (&#39;Код_группы&#39;, &#39;Наличие_Отца&#39;),
 (&#39;Код_группы&#39;, &#39;Опекунство&#39;),
 (&#39;Код_группы&#39;, &#39;Село&#39;),
 (&#39;Код_группы&#39;, &#39;Иностранец&#39;),
 (&#39;Код_группы&#39;, &#39;КодФакультета&#39;),
 (&#39;Код_группы&#39;, &#39;СрБаллАттестата&#39;),
 (&#39;Код_группы&#39;, &#39;male&#39;),
 (&#39;Код_группы&#39;, &#39;female&#39;),
 (&#39;Код_группы&#39;, &#39;sex_summ&#39;),
 (&#39;Код_группы&#39;, &#39;birth_year_int&#39;),
 (&#39;Код_группы&#39;, &#39;basis&#39;),
 (&#39;Код_группы&#39;, &#39;language&#39;),
 (&#39;Код_группы&#39;, &#39;country&#39;),
 (&#39;Код_группы&#39;, &#39;region&#39;),
 (&#39;Код_группы&#39;, &#39;city&#39;),
 (&#39;Код_группы&#39;, &#39;parents_country&#39;),
 (&#39;Год_Поступлени

In [10]:
combinations_frame = pd.DataFrame(all_combinations, columns=range(elements_num))
combinations_frame


,0,1
0,Код_группы,Год_Поступления
1,Код_группы,Год_Окончания_УЗ
2,Код_группы,Пособие
3,Код_группы,Общежитие
4,Код_группы,Наличие_Матери
...,...,...
226,country,city
227,country,parents_country
228,region,city
229,region,parents_country


In [11]:
def get_accuracy(frame_to_train):
    train, test = train_test_split(frame_to_train, test_size=0.20)
    
    train_input = train.drop(control_col, axis=1)    
    train_output = train[control_col]

    test_input = test.drop(control_col, axis=1)
    test_output = test[control_col]

    clf = RandomForestClassifier(random_state=0)
    clf.fit(train_input, train_output)
    pred = clf.predict(test_input)
    f1 = f1_score(test_output, pred, average='macro', zero_division = 0)
    vac_f1 = classification_report(test_output, pred, output_dict=True)['1']['f1-score']
    return vac_f1

In [12]:
def itter_from_frame(row):
    frame_columns = row.dropna().to_list()
    frame_columns.append(control_col)
    frame_to_train = base_df.loc[:, frame_columns]
    return get_accuracy(frame_to_train)

In [13]:

start, stop = 0, comb_size
combinations_frame.loc[start:stop, 'accuracy'] = combinations_frame.iloc[start:stop, :].apply(itter_from_frame, axis=1)

In [14]:
combinations_frame.head(30)

,0,1,accuracy
0,Код_группы,Год_Поступления,0.478261
1,Код_группы,Год_Окончания_УЗ,0.495050
2,Код_группы,Пособие,0.530973
3,Код_группы,Общежитие,0.515556
4,Код_группы,Наличие_Матери,0.566667
5,Код_группы,Наличие_Отца,0.466667
6,Код_группы,Опекунство,0.516432
7,Код_группы,Село,0.574074
8,Код_группы,Иностранец,0.495238
9,Код_группы,КодФакультета,0.467290


In [15]:
combinations_frame['accuracy'].describe()

count    231.000000
mean       0.056845
std        0.141917
min        0.000000
25%        0.000000
50%        0.000000
75%        0.028073
max        0.574074
Name: accuracy, dtype: float64

In [16]:
rt = combinations_frame.sort_values('accuracy', ascending=False)
rt.to_csv(f'data/combinations/columns_from_{min_col}_to_{elements_num}.csv')


In [57]:
columns = ['Код_группы', 'Год_Окончания_УЗ', '']

In [58]:
frame_columns = columns
frame_columns.append(control_col)
frame_to_train = base_df.loc[:, columns]

In [59]:
train, test = train_test_split(frame_to_train, test_size=0.20)

train_input = train.drop(control_col, axis=1)    
train_output = train[control_col]

test_input = test.drop(control_col, axis=1)
test_output = test[control_col]

clf = RandomForestClassifier(random_state=0)
clf.fit(train_input, train_output)
pred = clf.predict(test_input)
f1_score(test_output, pred, labels=[0,1], average='macro', zero_division = 0)

0.7150665508895125

In [60]:
report = classification_report(test_output, pred, output_dict=False)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2579
           1       0.57      0.38      0.45       138

    accuracy                           0.95      2717
   macro avg       0.77      0.68      0.72      2717
weighted avg       0.95      0.95      0.95      2717



In [40]:
report = classification_report(test_output, pred, output_dict=True)
print(report['1']['f1-score'])

0.4782608695652174


In [41]:
classification_report(test_output, pred, output_dict=True)['1']['f1-score']

0.4782608695652174

In [53]:
combinations_frame.groupby([0]).mean().agg([lambda x: x < 0.2])


,accuracy
,<lambda>
0,
basis,True
birth_year_int,True
city,True
country,True
female,True
language,True
male,True
region,True


In [67]:
combinations_frame.groupby([1]).max().agg([lambda x: x > 0.5])

,accuracy
,<lambda>
1,
basis,False
birth_year_int,False
city,False
country,True
female,True
language,True
male,True
parents_country,False
